In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from scipy.interpolate import CubicSpline
from sklearn.preprocessing import LabelEncoder

In [2]:
all_data = pd.read_csv("spline-test.csv", index_col=2)
ct_data = all_data['CT'].values #drop the /N values instead
age_data = all_data['PatientAge'].values
print(ct_data)

[23.95 33.65 27.28 31.22 26.8  17.34 31.08 29.66 24.8  32.4  23.6  28.4 ]


In [3]:
X = pd.DataFrame()
X['age'] = age_data
X['ct'] = ct_data
print(X)

          age     ct
0   15.000000  23.95
1   17.000000  33.65
2    8.000000  27.28
3    7.000000  31.22
4    0.666667  26.80
5    6.000000  17.34
6    8.000000  31.08
7   15.000000  29.66
8    5.000000  24.80
9   10.000000  32.40
10  10.000000  23.60
11   6.000000  28.40


In [17]:
def cubic(u):
    t = np.maximum(0, u)
    return t * t * t
    
# Generate cubic spline basis functions
def cubic_spline_basis_functions(x, knots):
    k0 = knots[0]
    k1 = knots[1]
    k2 = knots[2]
    c = (k2 - k0) * (k2 - k0)
    spline_values = ((cubic(x - k0) - cubic(x - k1) * (k2 - k0)/(k2 - k1) + cubic(x - k2) * (k1 - k0)/(k2 - k1)) / c )
    
    # spline_functions = [CubicSpline(knots, np.eye(len(knots))[i], extrapolate=False) for i in range(len(knots))]
    # spline_values = np.column_stack([f(x) for f in spline_functions])
    # print("x", x)
    # print("knots", knots)
    # print("splines", spline_values)
    # print("*********")
    return spline_values

In [18]:
# Knots for restricted cubic splines
age_knots = [2, 10, 16]
ct_knots = [18.6, 25.2, 34.5]

# Apply cubic spline transformations to variables
age_spline = cubic_spline_basis_functions(age_data, age_knots)
ct_spline = cubic_spline_basis_functions(ct_data, ct_knots)

# print(age_spline)
# print(ct_spline)

In [21]:
age_coeff0 = -0.3806
age_coeff1 = 0.3034

ct_coeff0 = -0.2139
ct_coeff1 = 0.1545

In [19]:
age_term = age_coeff0 * age_data + age_coeff1 * age_spline
print(age_term)

[-2.75962177 -2.48265714 -2.71044082 -2.4707051  -0.25373333 -2.18453061
 -2.71044082 -2.75962177 -1.8612051  -3.0134449  -3.0134449  -2.18453061]


In [68]:
# Stack spline-transformed variables with original variables
n = len(age_data)

X = np.hstack((age_data, ct_data))
X_with_splines = np.hstack((age_spline, ct_spline))
spline_coff = [age_coeff1] * n + [ct_coeff1] * n

linear_coff = [age_coeff0] * n + [ct_coeff0] * n

spline_term = spline_coff * X_with_splines

#print(X_with_splines)
#print(spline_term)

linear_term = linear_coff * X
#print(linear_term)

total_term = linear_term + spline_term

print(total_term)

[-2.75962177 -2.48265714 -2.71044082 -2.4707051  -0.25373333 -2.18453061
 -2.71044082 -2.75962177 -1.8612051  -3.0134449  -3.0134449  -2.18453061
 -5.0293223  -5.74487714 -5.44493188 -5.67758491 -5.39984164 -3.709026
 -5.67252983 -5.61016991 -5.1590704  -5.71424722 -4.97164864 -5.5338057 ]


In [41]:
np.shape(X_with_splines)

(24,)

In [43]:
np.shape(age_data)

(12,)

In [88]:
# Predefined weights and bias
weights = np.array([[-0.3806], [0.3034], [-0.2139], [0.1545]])

# data1 = np.array([age_data, age_spline, ct_data, ct_spline])
# data = data1.reshape(12,4)

data = np.column_stack((age_data, age_spline, ct_data, ct_spline))

data1 = np.column_stack((age_data, ct_data))

#model.layers[0].set_weights([weights])
#standard_bias = np.array([0]).reshape(1,)

#print(data1)
# print(weights)
# print(data)

log_reg = np.dot(data, weights)
print(log_reg)

[[-7.78894407]
 [-8.22753428]
 [-8.1553727 ]
 [-8.14829001]
 [-5.65357497]
 [-5.89355661]
 [-8.38297065]
 [-8.36979168]
 [-7.0202755 ]
 [-8.72769212]
 [-7.98509354]
 [-7.71833632]]


In [92]:
print(data.shape)
print(weights.shape)

(12, 4)
(4, 1)


In [89]:
# Create TensorFlow model
model = tf.keras.Sequential()

dense_layer = Dense(1, use_bias=False, input_shape=data.shape)

model.add(dense_layer)

model.compile(loss = "binary_crossentropy")
model.intercept = 6.5164
model.build()

dense_layer.set_weights([weights])

model.summary()

pred = model.predict(data)

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 12, 1)             4         
                                                                 
Total params: 4 (16.00 Byte)
Trainable params: 4 (16.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


ValueError: in user code:

    File "/Users/andres/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "/Users/andres/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/Users/andres/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "/Users/andres/anaconda3/lib/python3.10/site-packages/keras/src/engine/training.py", line 2283, in predict_step
        return self(x, training=False)
    File "/Users/andres/anaconda3/lib/python3.10/site-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/Users/andres/anaconda3/lib/python3.10/site-packages/keras/src/engine/input_spec.py", li

In [17]:
# Create TensorFlow model
model = tf.keras.Sequential()
# Add input layer
#model.add(Input(shape=(X_with_splines.shape[1],)))
# Add dense layer with predefined weights and bias
dense_layer = Dense(1, use_bias=False, input_shape=(X_with_splines.shape[1],))
model.add(dense_layer)
model.compile(loss = "binary_crossentropy")
model.intercept = 6.5164
model.build(input_shape=(None, X_with_splines.shape[1]))
# Set predefined weights
#model.layers[1].set_weights([weights])
dense_layer.set_weights([weights])
# Add more layers if needed
#model.add(Dense(64, activation=‘relu’))
#model.add(Dense(1, activation=‘sigmoid’))
# Print the model summary
model.summary()
#print(X_with_splines)
pred = model.predict(X_with_splines)
print(pred)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 11        
                                                                 
Total params: 11 (44.00 Byte)
Trainable params: 11 (44.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
1/1 [==============================] - 0s 35ms/step
[[15.070935]
 [      nan]
 [13.767315]
 [14.376268]
 [      nan]
 [      nan]
 [14.722734]
 [17.14024 ]
 [11.27151 ]
 [16.01176 ]
 [12.97714 ]
 [12.962517]]
